In [87]:
# -*- coding: utf-8 -*-

In [86]:
import os
import pandas as pd 

## tag information

In [6]:
tagFile = pd.read_excel('./data/Korean_POS_tags_comparison.xlsx',sheet_name='chart3',dtype = str)

In [ ]:
tagDict = dict()
for idx in tagFile.index:
    if tagFile.loc[idx]['Hannanum'][:3] =='XSN':
        tag = 'XSN'
    else:
        tag = tagFile.loc[idx]['Hannanum']
    tagDict[tag]=tagFile.loc[idx]['Twitter_Korean_Text']
    

In [ ]:
def TagChange(tag):
    return tagDict[tag]

## Add From NIA Dictionary

In [ ]:
niaDictFile = pd.read_excel('./data/NIADic.xlsx')

In [ ]:
niaDictFile = niaDictFile[niaDictFile['tag']!='xp']
niaDictFile.loc[:,'tag'] = niaDictFile['tag'].map(lambda x: TagChange(x.upper()))

In [ ]:
for idx in niaDictFile.tag.unique():
    outfile = ('./data/'+'twitter_tagger_' + idx +'_TagChange_from_NIADic.txt')
    with open(outfile,'w') as f:
        outdata = niaDictFile[niaDictFile['tag']==idx]['term']
        outdata.to_csv(outfile,index=False)

## 언론사 리스트

> stopwords를 위해서

In [88]:
pressList = pd.read_excel('./data/presslist.xlsx')
with(open('./data/newspress.txt','w')) as f:
    f.write('\n'.join(set(pressList['언론사'].tolist())))

# 자연어 처리

In [89]:
from ckonlpy.tag import Twitter as ctwitter
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression
import numpy as np
import matplotlib.pyplot as plt
from konlpy.tag import Twitter
from konlpy.utils import pprint

def Stopwords(file):
    stopwords = open(file,'r',  encoding='utf-8').readlines()
    stopwords = list(map(lambda x:x.strip(), stopwords))
    return stopwords
stopwordList = Stopwords('./data/newspress.txt')

In [90]:
ct = ctwitter()
#ot = Twitter()
#ct = Twitter()
xxxx = ct._dictionary._pos2words
list(map(lambda x: len(xxxx[x]), xxxx))

[776, 766754, 22658]

In [91]:
import sys
from collections import defaultdict
from datetime import datetime, timedelta
import time, re, pickle, itertools
import urllib3, json
sys.path.append('/Users/hyunyoun/Documents/GitHub/Private_Project/personal_project/')


In [92]:
import nltk
from __future__ import print_function, unicode_literals
from gensim.models.word2vec import Word2Vec
from sklearn.preprocessing import normalize as sknorm
import numpy as np
from operator import itemgetter
from krwordrank.word import KRWordRank
from krwordrank.hangle import normalize

In [93]:
import chat_bot as cb
import Database_Handler as dh

## 형태소 from ETRI AI

In [94]:
def OpenAPI():
    import pickle
    openapi = pickle.load(open('./etri_open_api_access_key', 'rb'))
    return openapi
def USE_ETRI_ANALYSIS(analysisCode, text):    
    openApiURL = "http://aiopen.etri.re.kr:8000/WiseNLU"
    accessKey = OpenAPI().strip()
    requestJson = {
        'access_key' : accessKey,
        'argument' : {
            'text' : text,
            'analysis_code' : analysisCode
        }
    }
    http = urllib3.PoolManager()
    response = http.request(
        "POST",
        openApiURL,
        headers={"Content-Type": "application/json; charset=UTF-8"},
        body=json.dumps(requestJson)
    )
    return json.loads(str(response.data,'utf-8')), str(response.status)

def USE_ETRI_INFO(fWord, sWord):
    openApiURL = "http://aiopen.etri.re.kr:8000/WiseWWN/WordRel"
    accessKey = OpenAPI().strip()
    firstWord = fWord
    #firstSenseId = fSenseId
    secondWord = sWord
    #secondSenseId = sSenseId
    requestJson = {
        "access_key": accessKey,
        "argument": {
        'first_word': firstWord,
        #'first_sense_id': firstSenseId,
        'second_word': secondWord,
        #'second_sense_id': secondSenseId
        }}
    http = urllib3.PoolManager()
    response = http.request("POST",openApiURL,headers={"Content-Type": "application/json; charset=UTF-8"},
    body=json.dumps(requestJson))
    return json.loads(str(response.data, 'utf-8')), str(response.status)

def USE_ETRI_RELATION(word):
    openApiURL = "http://aiopen.etri.re.kr:8000/WiseWWN/Word"
    accessKey = OpenAPI().strip()
    word = word
    requestJson = {
    "access_key": accessKey,
    "argument": {"word": word}}
    http = urllib3.PoolManager()
    response = http.request("POST",openApiURL,headers={"Content-Type": "application/json; charset=UTF-8"},
                            body=json.dumps(requestJson))
    return json.loads(str(response.data, 'utf-8')), str(response.status)

def Extract_Text_Info(idx):
    morp = list(map(lambda x: (x['lemma'], x['type']),idx['morp']))
    wsd = list(map(lambda x: (x['text'], x['type']),idx['WSD']))
    word = list(map(lambda x: (x['text'], x['type']),idx['word']))
    ne = list(map(lambda x: (x['text'], x['type']),idx['NE']))
    return morp, wsd, word, ne

def Run_ETRI_Analysis(stopwordsList, text):
    etri = USE_ETRI_ANALYSIS('srl', text)['return_object']    
    etri = etri['sentence']
    morp = [] ; wsd = [] ; word = [] ; ne = [] ; srl = [] ; dependency = []
    for i in etri:
        x = Extract_Text_Info(i)
        morp += x[0]
        wsd += x[1]
        word +=x[2]
        ne += x[3]
        for ii in i['SRL']:
            srl += list(map(lambda x: (x['text'], x['type']),ii['argument']))
        dependency += list(map(lambda x: (x['text'], x['label']), i['dependency']))
    return morp, wsd, word, ne, srl, dependency, etri
    
if __name__ == '__main__':
    pass

## TextRank 

In [95]:
import networkx
import re
 
class RawSentence:
    def __init__(self, textIter):
        if type(textIter) == str: self.textIter = textIter.split('\n')
        else: self.textIter = textIter
        self.rgxSplitter = re.compile('([.!?:](?:["\']|(?![0-9])))')
 
    def __iter__(self):
        for line in self.textIter:
            ch = self.rgxSplitter.split(line)
            for s in map(lambda a, b: a + b, ch[::2], ch[1::2]):
                if not s: continue
                yield s
 
class RawSentenceReader:
    def __init__(self, filepath):
        self.filepath = filepath
        self.rgxSplitter = re.compile('([.!?:](?:["\']|(?![0-9])))')
 
    def __iter__(self):
        for line in open(self.filepath, encoding='utf-8'):
            ch = self.rgxSplitter.split(line)
            for s in map(lambda a, b: a + b, ch[::2], ch[1::2]):
                if not s: continue
                yield s
 
class RawTagger:
    def __init__(self, textIter, tagger = None):
        if tagger:
            self.tagger = tagger
        else :
            from konlpy.tag import Komoran
            self.tagger = Komoran()
        if type(textIter) == str: self.textIter = textIter.split('\n')
        else: self.textIter = textIter
        self.rgxSplitter = re.compile('([.!?:](?:["\']|(?![0-9])))')
 
    def __iter__(self):
        for line in self.textIter:
            ch = self.rgxSplitter.split(line)
            for s in map(lambda a,b:a+b, ch[::2], ch[1::2]):
                if not s: continue
                yield self.tagger.pos(s)
 
class RawTaggerReader:
    def __init__(self, filepath, tagger = None):
        if tagger:
            self.tagger = tagger
        else :
            from konlpy.tag import Komoran
            self.tagger = Komoran()
        self.filepath = filepath
        self.rgxSplitter = re.compile('([.!?:](?:["\']|(?![0-9])))')
 
    def __iter__(self):
        for line in open(self.filepath, encoding='utf-8'):
            ch = self.rgxSplitter.split(line)
            for s in map(lambda a,b:a+b, ch[::2], ch[1::2]):
                if not s: continue
                yield self.tagger.pos(s)
 
class TextRank:
    def __init__(self, **kargs):
        self.graph = None
        self.window = kargs.get('window', 5)
        self.coef = kargs.get('coef', 1.0)
        self.threshold = kargs.get('threshold', 0.005)
        self.dictCount = {}
        self.dictBiCount = {}
        self.dictNear = {}
        self.nTotal = 0
 
 
    def load(self, sentenceIter, wordFilter = None):
        def insertPair(a, b):
            if a > b: a, b = b, a
            elif a == b: return
            self.dictBiCount[a, b] = self.dictBiCount.get((a, b), 0) + 1
 
        def insertNearPair(a, b):
            self.dictNear[a, b] = self.dictNear.get((a, b), 0) + 1
 
        for sent in sentenceIter:
            for i, word in enumerate(sent):
                if wordFilter and not wordFilter(word): continue
                self.dictCount[word] = self.dictCount.get(word, 0) + 1
                self.nTotal += 1
                if i - 1 >= 0 and (not wordFilter or wordFilter(sent[i-1])): insertNearPair(sent[i-1], word)
                if i + 1 < len(sent) and (not wordFilter or wordFilter(sent[i+1])): insertNearPair(word, sent[i+1])
                for j in range(i+1, min(i+self.window+1, len(sent))):
                    if wordFilter and not wordFilter(sent[j]): continue
                    if sent[j] != word: insertPair(word, sent[j])
 
    def loadSents(self, sentenceIter, tokenizer = None):
        import math
        def similarity(a, b):
            n = len(a.intersection(b))
            return n / float(len(a) + len(b) - n) / (math.log(len(a)+1) * math.log(len(b)+1))
 
        if not tokenizer: rgxSplitter = re.compile('[\\s.,:;-?!()"\']+')
        sentSet = []
        for sent in filter(None, sentenceIter):
            if type(sent) == str:
                if tokenizer: s = set(filter(None, tokenizer(sent)))
                else: s = set(filter(None, rgxSplitter.split(sent)))
            else: s = set(sent)
            if len(s) < 2: continue
            self.dictCount[len(self.dictCount)] = sent
            sentSet.append(s)
 
        for i in range(len(self.dictCount)):
            for j in range(i+1, len(self.dictCount)):
                s = similarity(sentSet[i], sentSet[j])
                if s < self.threshold: continue
                self.dictBiCount[i, j] = s
 
    def getPMI(self, a, b):
        import math
        co = self.dictNear.get((a, b), 0)
        if not co: return None
        return math.log(float(co) * self.nTotal / self.dictCount[a] / self.dictCount[b])
 
    def getI(self, a):
        import math
        if a not in self.dictCount: return None
        return math.log(self.nTotal / self.dictCount[a])
 
    def build(self):
        self.graph = networkx.Graph()
        self.graph.add_nodes_from(self.dictCount.keys())
        for (a, b), n in self.dictBiCount.items():
            self.graph.add_edge(a, b, weight=n*self.coef + (1-self.coef))
 
    def rank(self):
        return networkx.pagerank(self.graph, weight='weight')
 
    def extract(self, ratio = 0.1):
        ranks = self.rank()
        cand = sorted(ranks, key=ranks.get, reverse=True)[:int(len(ranks) * ratio)]
        pairness = {}
        startOf = {}
        tuples = {}
        for k in cand:
            tuples[(k,)] = self.getI(k) * ranks[k]
            for l in cand:
                if k == l: continue
                pmi = self.getPMI(k, l)
                if pmi: pairness[k, l] = pmi
 
        for (k, l) in sorted(pairness, key=pairness.get, reverse=True):
            print(k[0], l[0], pairness[k, l])
            if k not in startOf: startOf[k] = (k, l)
 
        for (k, l), v in pairness.items():
            pmis = v
            rs = ranks[k] * ranks[l]
            path = (k, l)
            tuples[path] = pmis / (len(path) - 1) * rs ** (1 / len(path)) * len(path)
            last = l
            while last in startOf and len(path) < 7:
                if last in path: break
                pmis += pairness[startOf[last]]
                last = startOf[last][1]
                rs *= ranks[last]
                path += (last,)
                tuples[path] = pmis / (len(path) - 1) * rs ** (1 / len(path)) * len(path)
 
        used = set()
        both = {}
        for k in sorted(tuples, key=tuples.get, reverse=True):
            if used.intersection(set(k)): continue
            both[k] = tuples[k]
            for w in k: used.add(w)
 
        #for k in cand:
        #    if k not in used or True: both[k] = ranks[k] * self.getI(k)
 
        return both
 
    def summarize(self, ratio = 0.333):
        r = self.rank()
        ks = sorted(r, key=r.get, reverse=True)[:int(len(r)*ratio)]
        return ' '.join(map(lambda k:self.dictCount[k], sorted(ks)))


In [96]:

stopwords =['이제','너무','중인' ,'만큼', '마찬가지', '꼬집었', "연합뉴스",'뉴스1','CBS노컷뉴스', "동아일보",
             "중앙일보", "조선일보", '기자','습니다','어요','이렇게', "아", "휴", "아이구", "아이쿠", "아이고",
             'OSEN','라고','지난','당시','오전','오후',"어", "나", "우리", "저희", "따라", "의해",'데일리',
             'JTBC','YTN','MBC','KBS','SBS','TVN','TV조선','채널A','MBN','이날','갑자기','같다','는데',
             '다른','에서','하는','하게','이후','통해','이날','정말','다시','스스로','같은','지난주','이번주','다음주',
             '그런','이런','첫째','둘째','셋째','넷째','다섯째','여섯째','일곱째','여덟째','아홉째','열째','기성',
             '이었','라는','저번','이번','면서','이렇게','저렇게','그렇게','정근','판청','아르테',
            '앵커','아나운서','좌윤','주고', 'B씨','A씨','경우','서로','대로','지사','인근','스완','위해','대한',
            '통한','기성','영제','간다','대해','수준','젊은','스탠','이씨','헤아','하라','패스트','좌안','이토','2TV',
            '성은','산다','나요','선우','누나','억원','태수','그냥','사이','어마','위엄','남자','로서','우리','누리','관련',
            '윤종','다친','승헌','골망','있는','까지','반종빈','녹이','현재','세이','카스티','스티','물론','라덴','상대로',
            '여러','아미','아사다','이닝','유기','신서','티브이데일리','공동','윤이','치원','타고','박씨','루수','유력','타고',
            '하고','처음','김씨','연재','사람','다이어','이다','인근','니다','아직','이씨','탈린','윤계','이날',
            '이것','저것','그것','신잡','장영','고씨','박씨','사실','경정','손새','장의','미도','말했',
            '빈살','실제','분들','건너','최근','조씨','장씨','올해','내년','작년','지난해','여서','A군','B군',
            '고든','램지','진혁','심지어','D씨','최근','니다','협회','그리즈','미오','그것','메이','이재','윈터',
            '임오','레이먼','사는','잡고','이야','치치','있다','1호','2호','3호','4호','5호','6호','7호','8호','9호',
            '다양한','부분','심씨','트루시','트루','심씨','부분','전체','주씨','펀치','원투','A사','1차','2차','3차','4차',
            '5차','6차','7차','8차','9차','10차','계속','MK','매경닷컴','MK스포츠','처음','모현','들이','있는',
            '더욱','차장','부장','시점','000','달라','모든','구름','맑음','방향','같아','믹스','나인','윤균','모두',
            '라며','헤아','제트','라카','소속','1분','신잡','울스','일단','번째','어서','이지','원장','교수','선수',
            '안젤리나','브래드','드려','조원','둥지','빅리그','어떻','대답했','루프','트리플','빅토르','지금','때문','해성',
             '빠졌','그리팅','5일','점프','그룹','시즌','크게','없는'] + stopwordList

#stopwords = stopwordList + ['중인' ,'만큼', '마찬가지', '꼬집었', "연합뉴스", "데일리", "동아일보", "중앙일보", "조선일보", "기자"
#        ,"아", "휴", "아이구", "아이쿠", "아이고", "어", "나", "우리", "저희", "따라", "의해", "을", "를", "에", "의", "가"]

In [97]:
class SentenceTokenizer(object):
    def __init__(self):
        self.twitter = ct
        #self.twitter = ot
        self.stopwords = stopwords
    def text2sentences(self, text):
        sentences = re.sub('\. ','.\n', text).split('\n')
        for idx in range(0, len(sentences)):
            if len(sentences[idx]) <= 10:
                sentences[idx-1] += (' ' + sentences[idx])
                sentences[idx] = ''
        return sentences
    def get_nouns(self, sentences):
        nouns = []
        for sentence in sentences:
            if sentence is not '':
                nouns.append(' '.join([noun for noun in self.twitter.phrases(str(sentence))
                                                    if noun not in self.stopwords and len(noun) > 1]))
                nouns.append(' '.join([noun for noun in self.twitter.nouns(str(sentence))
                                                  if noun not in self.stopwords and len(noun) > 1]))
                nouns.append(' '.join([noun for noun in self.twitter.morphs(str(sentence))
                                                   if noun not in self.stopwords and len(noun) > 1]))
        return nouns
    
class GraphMatrix(object):
    def __init__(self):
        self.tfidf = TfidfVectorizer(ngram_range=(1,5),sublinear_tf=True, lowercase=False)
        self.cnt_vec = CountVectorizer(ngram_range=(1,5), lowercase=False)
        self.graph_sentence = []
    def build_sent_graph(self, sentence):
        tfidf_mat = self.tfidf.fit_transform(sentence).toarray()
        #tfidf_mat = self.tfidf.fit_transform(sentence).toarray()
        self.graph_sentence = np.dot(tfidf_mat, tfidf_mat.T)
        return self.graph_sentence
    def build_words_graph(self, sentence):
        cnt_vec_mat = sknorm(self.cnt_vec.fit_transform(sentence).toarray().astype(float), axis=0)
        vocab = self.cnt_vec.vocabulary_
        return np.dot(cnt_vec_mat.T, cnt_vec_mat), {vocab[word] : word for word in vocab}

class Rank(object):
    def get_ranks(self, graph, d=0.85): # d = damping factor
        A = graph
        matrix_size = A.shape[0]
        for id in range(matrix_size):
            A[id, id] = 0 # diagonal 부분을 0으로
            link_sum = np.sum(A[:,id]) # A[:, id] = A[:][id]
            if link_sum != 0:
                A[:, id] /= link_sum
            A[:, id] *= -d
            A[id, id] = 1
        B = (1-d) * np.ones((matrix_size, 1))
        ranks = np.linalg.solve(A, B) # 연립방정식 Ax = b
        return {idx: r[0] for idx, r in enumerate(ranks)}

class TextRankX(object):
    def __init__(self, text):
        self.sent_tokenize = SentenceTokenizer()
        self.sentences = self.sent_tokenize.text2sentences(text)
        self.nouns = self.sent_tokenize.get_nouns(self.sentences)
        self.graph_matrix = GraphMatrix()
        self.sent_graph = self.graph_matrix.build_sent_graph(self.nouns)
        self.words_graph, self.idx2word = self.graph_matrix.build_words_graph(self.nouns)
        self.rank = Rank()
        self.sent_rank_idx = self.rank.get_ranks(self.sent_graph)
        self.sorted_sent_rank_idx = sorted(self.sent_rank_idx, key=lambda k: self.sent_rank_idx[k], reverse=True)
        self.word_rank_idx = self.rank.get_ranks(self.words_graph)
        self.sorted_word_rank_idx = sorted(self.word_rank_idx, key=lambda k: self.word_rank_idx[k], reverse=True)
    def summarize(self, sent_num=3):
        summary = []
        index=[]
        for idx in self.sorted_sent_rank_idx[:sent_num]:
            index.append(idx)
        index.sort()
        for idx in index:
            summary.append(self.sentences[idx])
        return summary
    def keywords(self, word_num=5):
        rank = Rank()
        rank_idx = rank.get_ranks(self.words_graph)
        sorted_rank_idx = sorted(rank_idx, key=lambda k: rank_idx[k], reverse=True)
        keywords = []
        index=[]
        for idx in sorted_rank_idx[:word_num]:
            index.append(idx)
        #index.sort()
        for idx in index:
            keywords.append(self.idx2word[idx])
        return keywords

# 분석

In [98]:
#site = 'Naver'
site = 'daum'
if site == 'daum':
    binList = []
    collection = 'newsDaum'
    etri_outcome = '/Users/hyunyoun/Documents/GitHub/Private_Project/personal_project/data/ETRI_OUTCOME/daum2/'
elif site.lower() == 'naver':
    binList = []
    collection = 'newsNaver'
    etri_outcome = '/Users/hyunyoun/Documents/GitHub/Private_Project/personal_project/data/ETRI_OUTCOME/naver2/'

mongodb = dh.ToMongoDB(*dh.AWS_MongoDB_Information())
dbname = 'hy_db'
useDb = dh.Use_Database(mongodb, dbname)
slack = cb.Slacker(cb.slacktoken())
useCollection = dh.Use_Collection(useDb, collection)
keywordList = list()
pressList = list()
for data in useCollection.find({'site':site}):
    pressList.append(data['press'])
    #if len(data['mainText'].strip()) == 0:
        #print (data['link'])
        #break
    idIs = data['_id']._ObjectId__id
    tis = data['title']+'. '+data['mainText']
    #tis2 = '.\n'.join(tis.split('. '))
    tis2 = data['mainText']
    if len(tis) != 0:
        if data['keywords'] !='NaN':
            keywordList += list(map(lambda x: ''.join(x.split(' · ')[-1].split(' ')), data['keywords']))
        else:
            pass
    
    
    print (data['link'])
    print (data['keywords'])
    #print (tis2)
    #print (tis2)
    try:
        from lexrankr import LexRank
        lexrank = LexRank()
        lexrank = LexRank(tagger='twitter')
        lexrank.summarize(tis2)
        lr = lexrank.probe(None)
    except:
        lr = tis2
    try:
        workrank = TextRankX(tis)
    except:
        try:
            workrank = TextRankX(tis)
        except:
            workrank = TextRankX(tis)
    print (workrank.keywords())
    print (lr)
    print ()
    '''
    if not os.path.isfile(etri_outcome+idIs.hex()+'.picked.txt'):
        try:
            etri = USE_ETRI_ANALYSIS('srl', tis2)
        except:
            print (etri[1])
            break
        else:
            pickle.dump(etri[0], open(etri_outcome+idIs.hex()+'.picked.txt','wb'))
            '''

http://v.media.daum.net/v/20171205151743503
['노홍철', '한명회', '한혜진', '종영', '편성시간']
['이름', '한명회', '출연자', '방송', '시간']
['이날은 특집 명에 맞게 녹화 전까지 철저히 출연자들이 서로 이름을 공유하지 못하도록 조치를 취했다', '그중 유독 눈에 띄는 한 남성 출연자의 기상천외한 이름이 밝혀져, 나머지 출연진은 물론 MC들도 당황하며 이름을 재차 물어 웃음을 자아냈다']

http://v.media.daum.net/v/20171209104014068
['류중일']
['캠프', '신인', '류중일 감독', '류중일', '감독']
['류중일 감독은 8일 "캠프 명단은 지난달 29일 전체 미팅을 하면서 통보를 했다', '류중일 감독은 "현재로서는 신인은 없다']

http://v.media.daum.net/v/20171205061623329
['불륜', '비', '첫방', '최다니엘', '최대철']
['보스', '비서', '상무', '좌윤이', '이는']
['좌윤이는 연인 경준(성훈 분)과 달달한 첫날밤을 보내려다가 보스 봉상무와 봉상무 아내(정영주 분), 일명 머저리와 미저리 부부의 전화폭격으로 분위기를 망쳤다', '좌윤이는 바람난 봉상무의 알리바이를 만들기 위해 화장품이 묻은 셔츠를 수습하느라 분주했고, 갑자기 내린 비에 발길을 서두르다가 와이퍼가 고장난 남치원(최다니엘 분)의 차에 치이며 무릎을 다쳤다']

http://v.media.daum.net/v/20171205140114189
['기억의밤', '시나리오', '리얼리티', '강하늘', '김무열']
['기억', '감독', '스릴러', '장항준 감독', '장항준']
['‘충분히 현실적인, 리얼리티 넘치는’ 추적 스릴러의 탄생을 위해 장항준 감독은 국과수는 물론, 정신분석학자 등 전문가들의 도움도 받았다', '장항준 감독은 ‘기억의 밤’의 포인트에 대해 “스릴과 서스펜스로 달려가는 막다른 종착역에 우리들의 슬픔이 있다고 말하고 싶다

['이영학', '변호사', '사건', '변호', '재판']
['(웃음) 아버지께선 ‘걔(이영학)는 똑똑해서 너도 속여먹을 수 있을 것 같더라’며 반대하셨고, 친구들도 “사건이 없어 배가 고픈 것도 아니면서 왜 그러냐’고 걱정하더라.” 김 변호사는 막상 사건을 맡은 뒤 걱정이 이만저만이 아니었다고 고백했다', '평생을 인권 변호사로 살다 간 고 조영래 변호사를 존경한다는 그는 “범죄사실만 놓고 보면 이영학이 ‘짐승만도 못한 사람’이라는 평가를 받을 수 있지만 그도 대한민국 국민”이라고 말했다']

http://v.media.daum.net/v/20171205140624398
['이슈 · 영흥도 낚싯배 전복사고', '문재인', '박근혜', '세월호']
['대통령', '구조', '사고', '세월호', '상황']
['이 답변서에서 박 전 대통령은 오전 10시께 국가안보실로부터 세월호 사고 첫 보고를 받고 오전 10시15분 김장수 당시 국가안보실장에게 전화해 “단 한명의 인명피해도 발생하지 않도록 (구조에 만전을 기)할 것, 여객선 내 객실 등을 철저히 확인하여 누락 인원이 없도록 할 것”을 지시했다고 주장했습니다', '결국 박 전 대통령의 세월호 관련 첫 발언은 오전 10시께 첫 보고 뒤 7시간이 지난 오후 5시15분께 중대본 방문 당시 발언으로 봐야 합니다']

http://v.media.daum.net/v/20171201140855351
NaN
['리즈', '리그', 'LG', '관심', '영입']
[' 메이저리그 구단의 한 스카우트는 “지금 리즈 구위와 전체적인 기량을 놓고 봤을 때, KBO리그 시절보다 훨씬 좋아졌다는 느낌을 받는다"며 "LG 트윈스를 비롯해 적지 않은 구단이 리즈 영입에 큰 관심을 두는 것으로 안다”고 전했다', '실제 수도권 프로구단 스카우트는 “도미니칸 윈터리그에서 가장 뜨거운 선수를 꼽자면 단연 리즈다']

http://v.media.daum.net/v/20171205090100814
['비만', '체중', '다이어트']
['시간', '

['김장', '장신영', '강경준', '시어머니', '예비 시어머니']
['지난주, 평소 함께 취미로 요리를 배우러 다니던 예비 시어머니에게 먼저 김장을 제안한 장신영은 가족들과 나눠먹기 위해 김치 ‘40포기’ 만들기에 도전하는 모습으로 관심을 모았다', '김장 경험이 있는 예비 시어머니의 주도 하에 찹쌀 풀 쑤기, 무채 썰기, 양파 갈기 등 각자 역할을 나눠 맡게 된 세 사람은 파이팅을 다지며 재료 준비를 시작했다']

http://v.media.daum.net/v/20171205102425927
['강하늘', '김무열', '기억의밤', 'tv드라마', '살인사건']
['감독', '장항준 감독', '장항준', '드라마', '자신']
['"드라마가 왜 명문대 출신 PD들을 뽑는지 알겠더라"고 말문을 연 장항준 감독은 "잠을 안 자고 공부해 본 사람들이다', '방송에 쫓기게 되면 그 피드백을 보면서 작가도 대본을 쓰고 감독도 연출을 하게 되더라"고 말했다']

http://v.media.daum.net/v/20171207215944021
['이승엽']
['이승엽', '야구선수', '이승엽 야구선수', '야구', '천직']
['[이승엽/전 야구선수 : 다 연관이 있는 것 같은데 굳이 꼽으시라면 저는 3번을 꼽고 싶습니다.] [앵커] 천직의식', '[이승엽/전 야구선수 : 천직의식과.] [앵커] 하나 더 해야 합니다']

http://v.media.daum.net/v/20171203003803154
['캠핑', '살짝미쳐도좋아', '눈길', '소녀시대', '반려견']
['효연', '캠핑', '공개', '데뷔', '소녀시대']
["3일 방송된 SBS '살짝 미쳐도 좋아'에서는 소녀시대 효연이 캠핑에 미쳐있는 ‘캠핑 미스타’로 출연했다", '효연은 “혼자 캠핑 장비를 들고 다닌 건 5~6년 됐다']

http://v.media.daum.net/v/20171204165700272
['안희정', '문재인', '경선']
['대통령', '발언', '지사', '지지자', '지사

['구조', '대통령', '현장', '오전', '사항']
['문 대통령은 곧바로 해경·행안부·세종상황실 등을 화상으로 연결해 상세보고를 받은 뒤 오전 9시 31분 국가위기관리센터에 구조작업 전반에 대한 사항을 지시했다', '문 대통령은 “지금 현재 총력을 다하고 있는데 그래도 정부가 추가로 지원할 것이 있으면 현장에 가서 상황을 파악하고 건의하라”고 김부겸 행안부 장관에게 별도 지시했다']

http://v.media.daum.net/v/20171203190258781
['며느리', '성폭력범죄', '낙태']
['며느리', '성폭력', '성폭행', '중형', '아들']
['의정부지법 형사합의12부(부장판사 노태선)는 성폭력 범죄의 처벌 등에 관한 특례법상 친족 관계에 의한 강간 등의 혐의로 기소된 이모(70)씨에게 징역 7년을 선고했다고 3일 밝혔다', '이씨는 2015년 함께 살던 아들이 숨진 지 불과 며칠이 지나지 않은 시점부터 며느리를 성폭행하려고 시도했다']

http://v.media.daum.net/v/20171203100441479
['조쉬 린드블럼']
['린드블럼', '롯데', '구단', '연봉', '협상']
['기본적으로 린드블럼은 계약 관계를 떠나 우리가 아끼는 선수다', '롯데는 린드블럼이 잘해도 연말 재계약 협상에서 큰 어려움이 없을 것으로 봤다']

http://v.media.daum.net/v/20171201060029695
['황재균', '윤석민']
['황재균', '윤석민', 'kt', '지명타자', '감독']
['프로 입단 당시 3루 수비를 주로 했던 윤석민은 트레이드 전 몸담았던 넥센에서는 주로 1루수나 지명타자로 출장했다', '김 감독은 "올 시즌 도중 윤석민을 트레이드로 데려오면서 취약한 포지션이 조금 메워졌다"면서도 "황재균과 달리 윤석민은 전문 3루수가 아니다"고 했다']

http://v.media.daum.net/v/20171206161349937
['감기몸살', '눈길', '웹툰', '라면']
['다니엘', '스케줄'

['온유', '사과', '팬들', '개월', '샤이니']
['이어 “지난 4개월 동안 활동을 쉬면서 부족한 저를 아껴주신 팬 여러분께 얼마나 큰 실망을 드렸는지, 깊이 반성하고 돌아보게 되었고, 제 스스로를 끝없이 원망하고 자책하기도 했다”는 온유는 “많이 사랑 받고 주목을 받을수록 더 철저하게 사적인 시간에도 책임감 있게 행동했어야 했는데, 실망스러운 모습을 보여드려 죄송한 마음뿐”이라고 밝혔습니다', '아직 검찰 결과가 나오지 않았다고는 해도 본인의 잘못을 인정하고 팬들에게 사과를 건네는 것은 사건 직후 충분히 할 수 있는 일이었죠']

http://v.media.daum.net/v/20171209132938485
['류현진']
['다저스', '트레이드', '류현진', '미팅', '거닉']
['역시 내년에 1천760만 달러(192억7천200만원)를 받는 왼손 투수 스콧 카즈미어도 다저스에 부담을 준다', '거닉 기자는 내년 다저스 전력에 포함될지 미지수인 곤살레스와 카즈미어가 트레이드 매물이 될 가능성은 작다면서 대신 류현진, 브랜던 매카시, 브록 스튜어트, 로스 스트리플링이 트레이드 카드가 될 수 있다고 예상했다']

http://v.media.daum.net/v/20171201225254745
['한혜진', '결별', '차우찬', '전현무', '박나래']
['한혜진', '기사', '결별', '전현무', '박나래']
['이날 박나래는 얼마 전 화제가 됐던 한혜진과 차우찬의 결벌 기사를 언급하자 한혜진은 발끈하며 "난 여자도 때려"라고 농담해 폭소를 유발했다', '박나래는 "영부인 말고 회장(전현무) 부인은 어떠냐"고 말했고 전현무는 "결별 후에 왜 내 기사가 그렇게 난 거냐']

http://v.media.daum.net/v/20171211223633282
['고든램지', '이연복', '오승환', '대결', '냉장고를부탁해']
['고든', '이연', '램지', '고든 램지', '대결']
["고든 램지는 '퀵 차돌박이 볶음 말이'를 만들었다", '특히 고든

['일본', '감독', '월드컵', '16', '콜롬비아']
['이어 트루시에 감독은 "원래 월드컵은 첫 번째 경기가 가장 중요하다"며, "일본은 반드시 콜롬비아를 이겨야 한다', '아울러 트루시에 감독은 "세네갈에는 사디오 마네(리버풀)만 있는 게 아니다"라며, "세네갈 선수들은 유럽 정상급 무대에서 활약 중이다']

http://v.media.daum.net/v/20171210014852769
['손흥민']
['손흥민', '토트넘', '전반', '경기', '스토크']
['손흥민이 공격을 주도한 토트넘이 전반을 지배했다', '◆ 손흥민 3경기 연속 골, 토트넘 스토크 대파 후반에도 손흥민의 활약은 여전했다']

http://v.media.daum.net/v/20171205071527845
['샤이니', '이글', '청춘시대', 'sm엔터테인먼트', '사과문']
['온유', '실망', '여러분', '자책', '시간']
['샤이니 온유는 12월 4일 오후 샤이니 공식 홈페이지에 "온유입니다"라는 제목으로 자필 사과문을 게재했다', '온유는 이 글에서 "지난 4개월 동안 쉬면서 깊이 반성하고 돌아보게 됐고 스스로를 원망하고 자책했다"며 "더 철저하게 사적인 시간에도 책임감 있게 행동했어야 했는데 실망스러운 모습 보여드려 죄송한 마음 뿐"이라고 사과했다']

http://v.media.daum.net/v/20171201060037707
['이슈 · 국정원·군 정치개입 의혹', '국가정보원', '김병찬', '댓글공작']
['서장', '검찰', '국정원', '안씨', '증언']
['2012년 12월 당시 서울경찰청을 담당하던 안씨는 “김 서장과 통화하며 국정원 댓글 수사상황을 들었다”고 검찰에 진술한 것으로 알려졌다', '그러나 안씨는 4년이 지난 최근 검찰에서 진술을 번복했고, 검찰은 이를 바탕으로 김 서장을 공무상 비밀누설 혐의의 피의자 신분으로 수사선상에 올린 것이다']

http://v.media.daum.net/v/20171211075017579
['마녀사냥'

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "<ipython-input-98-31a5113a39dd>", line 48, in <module>
    workrank = TextRankX(tis)
  File "<ipython-input-97-1edc7036bb10>", line 66, in __init__
    self.word_rank_idx = self.rank.get_ranks(self.words_graph)
  File "<ipython-input-97-1edc7036bb10>", line 52, in get_ranks
    ranks = np.linalg.solve(A, B) # 연립방정식 Ax = b
  File "/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/numpy/linalg/linalg.py", line 375, in solve
    r = gufunc(a, b, signature=signature, extobj=extobj)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "<ipython-input-98-31a5113a39dd>", line 51, in <module>
    workrank = TextRankX(tis)
  File "<ipython-input-97-1edc7036bb10>", line 59, in __init__
    self.nouns = self.sent_tokenize.get_nouns(self.sentences)
  File "<ipython-input-97-1edc7036bb10>", line 19, in get_nouns
    nouns.append(' '.join([noun for noun in self.twitter.nouns(st

KeyboardInterrupt: 

In [85]:
import networkx
import re
 
class RawSentence:
    def __init__(self, textIter):
        if type(textIter) == str: self.textIter = textIter.split('\n')
        else: self.textIter = textIter
        self.rgxSplitter = re.compile('([.!?:](?:["\']|(?![0-9])))')
 
    def __iter__(self):
        for line in self.textIter:
            ch = self.rgxSplitter.split(line)
            for s in map(lambda a, b: a + b, ch[::2], ch[1::2]):
                if not s: continue
                yield s
 
class RawSentenceReader:
    def __init__(self, filepath):
        self.filepath = filepath
        self.rgxSplitter = re.compile('([.!?:](?:["\']|(?![0-9])))')
 
    def __iter__(self):
        for line in open(self.filepath, encoding='utf-8'):
            ch = self.rgxSplitter.split(line)
            for s in map(lambda a, b: a + b, ch[::2], ch[1::2]):
                if not s: continue
                yield s
 
class RawTagger:
    def __init__(self, textIter, tagger = None):
        if tagger:
            self.tagger = tagger
        else :
            from konlpy.tag import Komoran
            self.tagger = Komoran()
        if type(textIter) == str: self.textIter = textIter.split('\n')
        else: self.textIter = textIter
        self.rgxSplitter = re.compile('([.!?:](?:["\']|(?![0-9])))')
 
    def __iter__(self):
        for line in self.textIter:
            ch = self.rgxSplitter.split(line)
            for s in map(lambda a,b:a+b, ch[::2], ch[1::2]):
                if not s: continue
                yield self.tagger.pos(s)
 
class RawTaggerReader:
    def __init__(self, filepath, tagger = None):
        if tagger:
            self.tagger = tagger
        else :
            from konlpy.tag import Komoran
            self.tagger = Komoran()
        self.filepath = filepath
        self.rgxSplitter = re.compile('([.!?:](?:["\']|(?![0-9])))')
 
    def __iter__(self):
        for line in open(self.filepath, encoding='utf-8'):
            ch = self.rgxSplitter.split(line)
            for s in map(lambda a,b:a+b, ch[::2], ch[1::2]):
                if not s: continue
                yield self.tagger.pos(s)
 
class TextRank:
    def __init__(self, **kargs):
        self.graph = None
        self.window = kargs.get('window', 5)
        self.coef = kargs.get('coef', 1.0)
        self.threshold = kargs.get('threshold', 0.005)
        self.dictCount = {}
        self.dictBiCount = {}
        self.dictNear = {}
        self.nTotal = 0
 
 
    def load(self, sentenceIter, wordFilter = None):
        def insertPair(a, b):
            if a > b: a, b = b, a
            elif a == b: return
            self.dictBiCount[a, b] = self.dictBiCount.get((a, b), 0) + 1
 
        def insertNearPair(a, b):
            self.dictNear[a, b] = self.dictNear.get((a, b), 0) + 1
 
        for sent in sentenceIter:
            for i, word in enumerate(sent):
                if wordFilter and not wordFilter(word): continue
                self.dictCount[word] = self.dictCount.get(word, 0) + 1
                self.nTotal += 1
                if i - 1 >= 0 and (not wordFilter or wordFilter(sent[i-1])): insertNearPair(sent[i-1], word)
                if i + 1 < len(sent) and (not wordFilter or wordFilter(sent[i+1])): insertNearPair(word, sent[i+1])
                for j in range(i+1, min(i+self.window+1, len(sent))):
                    if wordFilter and not wordFilter(sent[j]): continue
                    if sent[j] != word: insertPair(word, sent[j])
 
    def loadSents(self, sentenceIter, tokenizer = None):
        import math
        def similarity(a, b):
            n = len(a.intersection(b))
            return n / float(len(a) + len(b) - n) / (math.log(len(a)+1) * math.log(len(b)+1))
 
        if not tokenizer: rgxSplitter = re.compile('[\\s.,:;-?!()"\']+')
        sentSet = []
        for sent in filter(None, sentenceIter):
            if type(sent) == str:
                if tokenizer: s = set(filter(None, tokenizer(sent)))
                else: s = set(filter(None, rgxSplitter.split(sent)))
            else: s = set(sent)
            if len(s) < 2: continue
            self.dictCount[len(self.dictCount)] = sent
            sentSet.append(s)
 
        for i in range(len(self.dictCount)):
            for j in range(i+1, len(self.dictCount)):
                s = similarity(sentSet[i], sentSet[j])
                if s < self.threshold: continue
                self.dictBiCount[i, j] = s
 
    def getPMI(self, a, b):
        import math
        co = self.dictNear.get((a, b), 0)
        if not co: return None
        return math.log(float(co) * self.nTotal / self.dictCount[a] / self.dictCount[b])
 
    def getI(self, a):
        import math
        if a not in self.dictCount: return None
        return math.log(self.nTotal / self.dictCount[a])
 
    def build(self):
        self.graph = networkx.Graph()
        self.graph.add_nodes_from(self.dictCount.keys())
        for (a, b), n in self.dictBiCount.items():
            self.graph.add_edge(a, b, weight=n*self.coef + (1-self.coef))
 
    def rank(self):
        return networkx.pagerank(self.graph, weight='weight')
 
    def extract(self, ratio = 0.1):
        ranks = self.rank()
        cand = sorted(ranks, key=ranks.get, reverse=True)[:int(len(ranks) * ratio)]
        pairness = {}
        startOf = {}
        tuples = {}
        for k in cand:
            tuples[(k,)] = self.getI(k) * ranks[k]
            for l in cand:
                if k == l: continue
                pmi = self.getPMI(k, l)
                if pmi: pairness[k, l] = pmi
 
        for (k, l) in sorted(pairness, key=pairness.get, reverse=True):
            print(k[0], l[0], pairness[k, l])
            if k not in startOf: startOf[k] = (k, l)
 
        for (k, l), v in pairness.items():
            pmis = v
            rs = ranks[k] * ranks[l]
            path = (k, l)
            tuples[path] = pmis / (len(path) - 1) * rs ** (1 / len(path)) * len(path)
            last = l
            while last in startOf and len(path) < 7:
                if last in path: break
                pmis += pairness[startOf[last]]
                last = startOf[last][1]
                rs *= ranks[last]
                path += (last,)
                tuples[path] = pmis / (len(path) - 1) * rs ** ( / len(path)) * len(path)
 
        used = set()
        both = {}
        for k in sorted(tuples, key=tuples.get, reverse=True):
            if used.intersection(set(k)): continue
            both[k] = tuples[k]
            for w in k: used.add(w)
 
        #for k in cand:
        #    if k not in used or True: both[k] = ranks[k] * self.getI(k)
 
        return both
 
    def summarize(self, ratio = 0.333):
        r = self.rank()
        ks = sorted(r, key=r.get, reverse=True)[:int(len(r)*ratio)]
        return ' '.join(map(lambda k:self.dictCount[k], sorted(ks)))


SyntaxError: invalid syntax (<ipython-input-85-581aa7cd2d59>, line 168)

In [ ]:
testdata = pickle.load(open(etri_outcome+data['_id']._ObjectId__id.hex()+'.picked.txt','rb'))
etri = testdata['return_object']['sentence']
morp = [] ; wsd = [] ; word = [] ; ne = [] ; srl = [] ; dependency = []

for i in etri:
    x = Extract_Text_Info(i)
    morp += x[0]
    wsd += x[1]
    word +=x[2]
    ne += x[3]
    for ii in i['SRL']:
        srl += list(map(lambda x: (x['text'], x['type']),ii['argument']))
    dependency += list(map(lambda x: (x['text'], x['label']), i['dependency']))
morp1 = list(map(lambda x: x[0], morp))
wsd1 = list(map(lambda x: x[0], wsd))
word1 = list(map(lambda x: x[0], word))
ne1 = list(map(lambda x: x[0], ne))
srl1 = list(map(lambda x: x[0], srl))
dependency1 = list(map(lambda x: x[0], dependency))
outlist = []
for idx in range(len(etri)):
    y = pd.DataFrame(etri[idx]['dependency'])
    y2 = y[y['mod'].apply(lambda x: len(x)) ==y['mod'].apply(lambda x: len(x)).max()]
    y3 = y2[y2['weight'] == y2['weight'].max()]
    y4 = y[y['id'].isin(y3['mod'][y3['mod'].index[0]])]
    y5 = y[y['weight'] ==y4['weight'].max()]
    y6 = y2.text.values.tolist() + y5.text.values.tolist()
    y7 = y[y.text.isin(y6)]    
    out = ' '.join(y7.text.tolist())
    outlist.append(out)

In [ ]:
sTest = data['title']+'. '+data['mainText']
sTest = '.\n'.join(sTest.split('. '))
sTest

In [ ]:
sTest2 = sTest.split('\n')
sTest2

In [ ]:
TextRank(re.sub('[\W]','',sTest)).keywords()

* 형태소 분석 : morp
* 어휘의미 분석 : wsd (동음이의어)
* 어휘의미 분석 : wsd_poly (다의어)
* 개체명 인식 : ner
* 의존구문 분석 : dependency
* 의미역 인식 : srl

In [ ]:
workrank1 = RunWordKRwordRank(data['title']+'. '+data['mainText'], 3)
workrank1[0]

In [ ]:
keyword(etri_outcome,data)

In [ ]:
class PMICalc:
    def __init__(self, **kargs):
        self.window = kargs.get('window', 5)
        self.minNum = kargs.get('minNum', 5)
        self.dictCount = {}
        self.dictBiCount = {}
        self.searchPair = {}
        self.nTotal = 0
 
    def train(self, sentenceIter):
        def insertPair(a, b):
            if a > b: a, b = b, a
            elif a == b: return
            self.dictBiCount[a, b] = self.dictBiCount.get((a, b), 0) + 1
            if a in self.searchPair: self.searchPair[a].add(b)
            else: self.searchPair[a] = set([b])
            if b in self.searchPair: self.searchPair[b].add(a)
            else: self.searchPair[b] = set([a])
 
        for sent in sentenceIter:
            self.nTotal += len(sent)
            for i, word in enumerate(sent):
                self.dictCount[word] = self.dictCount.get(word, 0) + 1
                for j in range(i+1, min(i+self.window+1, len(sent))):
                    if sent[j] != word: insertPair(word, sent[j])
 
    def getCoOccurrence(self, a, b):
        if a > b: a, b = b, a
        elif a == b: return
        return self.dictBiCount.get((a, b), 0)
 
    def getPMI(self, a, b):
        import math
        co = self.getCoOccurrence(a, b)
        if not co: return None
        return math.log(float(co) * self.nTotal / self.dictCount[a] / self.dictCount[b])
 
    def getHighestPair(self, a, n = 10):
        return sorted(map(lambda b:(b, self.getPMI(a,b)), filter(lambda x:self.getCoOccurrence(a,x) >= self.minNum, self.searchPair[a])), key=lambda x:x[1], reverse=True)[:n]
# 행 별로 문장이 구분되어있고, 탭으로 단어가 구분된 텍스트 파일을 읽어줍니다.
class SentenceReader:
    def __init__(self, filepath):
        self.filepath = filepath
        self.rgxFilter = re.compile('.+/[NAVM].+')
 
    def __iter__(self):
        for line in open(self.filepath, encoding='utf-8'):
            yield list(s.split('\t'))

pc = PMICalc(window=8)
pc.train(ct.nouns(tis)) # test.txt 파일을 읽어와 분석합니다.
print(pc.getHighestPair('노홍철', 10)) # 사람/NNG과 관련성이 높은 단어 100개를 출력합니다.

In [ ]:
def Extract_Keywords(text, stopwords):
    etri = Run_ETRI_Analysis(stopwords, text)
    soy1 = RunLRNounExtractor(text)
    soy2 = RunWordExtractor(text)
    et0 = list(filter(lambda x: x[1] in ['NNG', 'NNP','NNG','SL','SH','NH'], etri[0]))
    et0 = list(map(lambda x: x[0], et0))
    et1 = list(filter(lambda x: x[1] in ['NNG', 'NNP','NNG','SL','SH','NH'], etri[1]))
    et1 = list(map(lambda x: x[0], et1))
    et2 = list(map(lambda x: x[0], etri[2]))
    et3 = list(map(lambda x: x[0], etri[3]))
    et4 = list(filter(lambda x: x[1] in ['ARG0','ARG1','ARG2','ARG3','ARG4'], etri[4]))
    et4 = list(map(lambda x: x[0], et4))
    et5 = list(filter(lambda x: x[1] in ['NP','NP_SBJ','NP_OBJ','NP_AJT','VNP'], etri[5]))
    et5 = list(map(lambda x: x[0], et5))
    mecabout = list(filter(lambda x: x[1] in ['NNG','NNB','NNP'], mecab.pos(text)))
    ctout = list(filter(lambda x: x[1] == 'Noun' , ct.pos(text)))
    otout = list(filter(lambda x: x[1] == 'Noun' , ot.pos(text)))
    mcout = list(map(lambda x: x[0], mecabout))
    ctout = list(map(lambda x: x[0], ctout))
    otout = list(map(lambda x: x[0], otout))
    out = list(soy1.keys())+list(soy2.keys())+ctout+otout+mcout+et3+et4+et0+et1+et2+et5
    vect = TfidfVectorizer().fit(out)
    y = [list(soy1.keys()), list(soy2.keys()),et3, et4, ctout, otout, mcout,et0, et1, et2, et5]
    y = list(filter(lambda x: len(x) !=0, y))
    outdict = dict()
    for i in y:
        count = vect.transform(i).toarray().sum(axis = 0)
        idx = np.argsort(-count)
        count = count[idx]
        feature_name = np.array(vect.get_feature_names())[idx]
        out = dict(zip(feature_name, count))
        for ii in out:
            if not ii in outdict:
                outdict[ii] = out[ii]
            else:
                outdict[ii] +=out[ii]
    x = sorted(outdict.items(), key = itemgetter(1), reverse=True)[:5]
    output = list(map(lambda x: x[0], x))
    return output


In [ ]:
def getPMI(self, a, b):
    import path
    co = self.dictNear.get((a,b), 0)
    if not co: return None
    return math.log(float(co) * self.nTotal / self.dictCount[a] / self.dictCount[b])
def iterate_textrank(self):
    converged = True
    for node in self.graph.nodes(data=True):
        old_score = node[1]['score']
        self.graph.add_node(node[0],
                            score=self.calculate_score(node[0]))
        if converged and abs(old_score - node[1]['score']) > self.convergence_threshold:
            converged = False
def calculate_score(self, node):
    score_from_neighbors = sum(map(lambda n: self.graph.node[n]['score'] / len(self.graph.neighbors(n)),
                                   self.graph.neighbors(node)))
    return (1-self.damping_factor) + self.damping_factor * score_from_neighbors

def textrank(self, max_iter = None, data = False):
    if max_iter is None:
        max_iter = self.graph.number_of_nodes()
    for _in in range(max_iter):
        if self.iterate_texterank():
            break
    sorted_nodes = sorted(self.graph.node(data = True), key= lambda node: node[1]['score'], reverse=True)
    if not data:
        sorted_nodes = list(mpa(lambda node:node[0], sorted_nodes))
    return sorted_nodes


In [ ]:
for (k, l), v in pairness.items():
    pmis = v
    rs= ranks[k] * ranks[l]
    path = (k,l)
    tuples[path] = pmis / (len(path)-1) * rs ** (1 / len(path))* len(path)
    last = l
    while last in startOf and len(path) < 7:
        if last in path:break
        pmis += pairness[startOf[last]]
        last = startOf[last][1]
        rs *=ranks[last]
        path += (last, )
        tuples[path] = pmis / (len(path) - 1) * rs ** (1 / len(path)) * len(path)

In [ ]:
from collections import Counter
import random

def p_topic_given_document(topic, d, alpha=0.1):
    return ((document_topic_counts[d][topic] + alpha) /
            (document_lengths[d] + K * alpha))

def p_word_given_topic(word, topic, beta=0.1):
    return ((topic_word_counts[topic][word] + beta) /
            (topic_counts[topic] + V * beta))

def topic_weight(d, word, k):
    return p_word_given_topic(word, k) * p_topic_given_document(k, d)

def choose_new_topic(d, word):
    return sample_from([topic_weight(d, word, k) for k in range(K)])

def sample_from(weights):
    total = sum(weights)
    rnd = total * random.random()
    for i, w in enumerate(weights):
        rnd -= w
        if rnd <= 0:
            return i
'''
documents = [["Hadoop", "Big Data", "HBase", "Java", "Spark", "Storm", "Cassandra"],
    ["NoSQL", "MongoDB", "Cassandra", "HBase", "Postgres"],
    ["Python", "scikit-learn", "scipy", "numpy", "statsmodels", "pandas"],
    ["R", "Python", "statistics", "regression", "probability"],
    ["machine learning", "regression", "decision trees", "libsvm"],
    ["Python", "R", "Java", "C++", "Haskell", "programming languages"],
    ["statistics", "probability", "mathematics", "theory"],
    ["machine learning", "scikit-learn", "Mahout", "neural networks"],
    ["neural networks", "deep learning", "Big Data", "artificial intelligence"],
    ["Hadoop", "Java", "MapReduce", "Big Data"],
    ["statistics", "R", "statsmodels"],
    ["C++", "deep learning", "artificial intelligence", "probability"],
    ["pandas", "R", "Python"],
    ["databases", "HBase", "Postgres", "MySQL", "MongoDB"],
    ["libsvm", "regression", "support vector machines"]]
'''
random.seed(0)
K=4
document_topics = [[random.randrange(K) for word in document]
                    for document in documents]
document_topic_counts = [Counter() for _ in documents]
topic_word_counts = [Counter() for _ in range(K)]
topic_counts = [0 for _ in range(K)]
document_lengths = [len(document) for document in documents]
distinct_words = set(word for document in documents for word in document)
V = len(distinct_words)
D = len(documents)

for d in range(D):
    for word, topic in zip(documents[d], document_topics[d]):
        document_topic_counts[d][topic] += 1
        topic_word_counts[topic][word] += 1
        topic_counts[topic] += 1

for iter in range(1000):
    for d in range(D):
        for i, (word, topic) in enumerate(zip(documents[d],
                                              document_topics[d])):
            document_topic_counts[d][topic] -= 1
            topic_word_counts[topic][word] -= 1
            topic_counts[topic] -= 1
            document_lengths[d] -= 1
            new_topic = choose_new_topic(d, word)
            document_topics[d][i] = new_topic
            document_topic_counts[d][new_topic] += 1
            topic_word_counts[new_topic][word] += 1
            topic_counts[new_topic] += 1
            document_lengths[d] += 1

# sentiment analysis

In [ ]:
import os
import sys

In [ ]:
def read_data(filename):
    with codecs.open(filename, encoding = 'utf-8', mode='r') as f:
        data